In [1]:
def keystream(B,S):
    L = B[7] + B[11] + B[30] + B[40] + B[45] + B[54] + B[71]
    Q = B[4]*B[21] + B[9]*B[52] + B[18]*B[37] + B[44]*B[76]
    T = B[5] + B[8]*B[82] + B[34]*B[67]*B[73] + B[2]*B[28]*B[41]*B[65] + B[13]*B[29]*B[50]*B[64]*B[75] \
    + B[6]*B[14]*B[26]*B[32]*B[47]*B[61] + B[1]*B[19]*B[27]*B[43]*B[57]*B[66]*B[78]
    T_c = S[23] + S[3]*S[16] + S[9]*S[13]*B[48] + S[1]*S[24]*B[38]*B[63]
    
    out = L + Q + T + T_c + b0 + b0
    return out

def NFSR1_bk_update(S):
    temp = S[30]  + S[1]  +  S[4]  + S[5]  +  S[14] + S[16] + S[17] + S[19] +  S[24] + S[7]*S[17] + S[7]*S[19] + S[11]*S[20] \
        + S[13]*S[18] + S[16]*S[20] + S[19]*S[21] + S[3]*S[11]*S[21] + S[3]*S[18]*S[21] + S[6]*S[19]*S[20] \
        + S[7]*S[17]*S[21] + S[7]*S[19]*S[21] + S[11]*S[18]*S[21] + S[19]*S[20]*S[21] + S[3]*S[6]*S[11]*S[20]\
        + S[3]*S[6]*S[18]*S[20] + S[3]*S[11]*S[20]*S[21] + S[3]*S[18]*S[20]*S[21] + S[6]*S[7]*S[17]*S[20] \
        + S[6]*S[7]*S[19]*S[20] + S[6]*S[11]*S[18]*S[20] + S[7]*S[17]*S[20]*S[21] + S[7]*S[19]*S[20]*S[21]\
        + S[11]*S[18]*S[20]*S[21] + b0 + b0
    
    S = [temp] + S[:30]
    return S

def NFSR2_bk_update(B,s0):
    temp = s0 + B[89] + B[23] + B[48] + B[78] + B[83] + B[2]*B[58] + B[9]*B[11] + B[14]*B[15] + B[24]*B[52] \
    + B[34]*B[41] + B[54]*B[57] + B[59]*B[73] + B[19]*B[21]*B[22] + B[61]*B[67]*B[71] \
    + B[76]*B[79]*B[80]*B[82] + b0 + b0
    
    B = [temp] + B[:89]
    return B
    
def NFSR1_fw_update(S):
    temp = S[0]  + S[2]  +  S[5]  + S[6]  +  S[15] + S[17] + S[18] + S[20] +  S[25] + S[8]*S[18] + S[8]*S[20] + S[12]*S[21] \
        + S[14]*S[19] + S[17]*S[21] + S[20]*S[22] + S[4]*S[12]*S[22] + S[4]*S[19]*S[22] + S[7]*S[20]*S[21] \
        + S[8]*S[18]*S[22] + S[8]*S[20]*S[22] + S[12]*S[19]*S[22] + S[20]*S[21]*S[22] + S[4]*S[7]*S[12]*S[21]\
        + S[4]*S[7]*S[19]*S[21] + S[4]*S[12]*S[21]*S[22] + S[4]*S[19]*S[21]*S[22] + S[7]*S[8]*S[18]*S[21] \
        + S[7]*S[8]*S[20]*S[21] + S[7]*S[12]*S[19]*S[21] + S[8]*S[18]*S[21]*S[22] + S[8]*S[20]*S[21]*S[22]\
        + S[12]*S[19]*S[21]*S[22] + b0 + b0
    
    for i in range(30):
        S[i]=S[i + 1]
    S[30]=temp
    return(S)

    
def NFSR2_fw_update(B,s0):
    temp = s0 + B[0] + B[24] + B[49] + B[79] + B[84] + B[3]*B[59] + B[10]*B[12] + B[15]*B[16] + B[25]*B[53] \
    + B[35]*B[42] + B[55]*B[58] + B[60]*B[74] + B[20]*B[22]*B[23] + B[62]*B[68]*B[72] \
    + B[77]*B[80]*B[81]*B[83] + b0 + b0
    
    for i in range(89):
        B[i]=B[i + 1]
    B[89]=temp
    return(B)


def intersection(A,B):
    temp = []
    for i in range(len(A)):
        if A[i] in B:
            temp.append(A[i])
    return temp


In [15]:
import time
import csv 
import numpy as np
import gurobipy as gp
from gurobipy import GRB

start2 = time.time()

filename = 'Lizard_TMDTO_bkd.txt'
csv_file = 'Lizard_TMDTO_bkd.csv'

for keys in range(5):
    keys_b = 0
    keys_f = keys-keys_b


    #pattern = [0]
    #for i in range(1,max(keys_f,keys_b)+1,1):
    #    if i < keys_b+1:
    #        pattern = pattern + [-i]
    #    if i < keys_f:
    #        pattern = pattern + [i]
    pattern = [-i for i in range(1,keys_b+1,1)] + [i for i in range(keys_f)]
    #pattern = [i for i in range(-keys_b,0,1)] + [i for i in range(keys_f)]

    print("Pattern of the keystream bits ", pattern)
    state_len = 121
    V = BooleanPolynomialRing(state_len,['b%d'%(i) for i in range(90)] + ['s%d'%(i) for i in range(31)])
    V.inject_variables()
    g = list(V.gens()) 

    #Assigning variables to the registers
    B = g[:90] 
    S = g[90:]

    R = []
    F = []
    G = []

    for i in F:
        if i < 90:
            B[i] = 0
        else:
            S[i-90] = 0



    linear_b = [0]*keys_b
    linear_f = [0]*keys_f
    n_mon = 0
    Z_f = [0]*keys_f
    Z_b = [0]*keys_b

    for loop in range(keys_b):
        S = NFSR1_bk_update(S)
        B = NFSR2_bk_update(B,S[0])

        Z_b[loop] = keystream(B,S)
        monom = list(Z_b[loop].monomials())
        linear_b[loop] = [m for m in monom if m.degree() == 1]
        for m in monom:
            if m.degree()>1:
                n_mon += 1
        #print("backward loop 1st part ", loop)

    B = g[:90] 
    S = g[90:]

    for i in F:
        if i < 90:
            B[i] = 0
        else:
            S[i-90] = 0


    for loop in range(keys_f):
        Z_f[loop] = keystream(B,S)
        monom = list(Z_f[loop].monomials())
        linear_f[loop] = [m for m in monom if m.degree() == 1]
        for m in monom:
            if m.degree()>1:
                n_mon += 1

        #print("forward loop 1st part ",loop)
        if loop != keys_f-1:
            B=NFSR2_fw_update(B,S[0])
            S=NFSR1_fw_update(S) 
    #print("1st part done")



    index = 0
    Z = [0]*(keys_f+keys_b)
    linear = [0]*(keys_f + keys_b)
    linear_new = [0]*(keys_f + keys_b)
    linear_all = []

    for i in pattern:
        if i < 0:
            Z[index] = Z_b[-i-1]
            linear[index] = linear_b[-i-1]
        if i >= 0:
            Z[index] = Z_f[i]
            linear[index] = linear_f[i]
        index += 1


    for loop in range(len(Z)):
        lin = []
        for i in linear[loop]:
            if g.index(i) not in linear_all:
                lin = lin + [g.index(i)]

        linear_new[loop] = lin
        linear_all = linear_all + lin

    #########Gurobi Modelling Starts###############
    model = gp.Model()


    v = [0,1,2]             # Temporary variable to be used for gurobi variable definition
    aux = [i for i in range(n_mon)]

    #Let D, F, G represents the class of determined/recovery, Fixed bit, and Guess bit
    #Since these are set and we need variables to denote R,F and G therefore let 2=R, 3=F and 4=G
    #x represents variables for each register and m contains 3 variables for each x[i] i.e. for R,F and G
    x = model.addVars(len(B) + len(S), lb = 2, ub = 4, vtype = GRB.INTEGER, name = "x")
    m = model.addVars(len(B) + len(S), len(v), vtype = GRB.BINARY, name = "m")
    q = model.addVars(len(aux), vtype = GRB.BINARY, name = "q")
    #p = model.addVars(len(aux), vtype = GRB.BINARY, name = "p")
    #r = model.addVars(len(aux), vtype = GRB.BINARY, name = "r")
    model.update()

    model.addConstrs((m[i,0] == 1) for i in R)
    model.addConstrs((m[i,1] == 1) for i in F)
    model.addConstrs((m[i,2] == 1) for i in G)

    #Objective function is to maximize recovery bit and minimize the number of FIXED bit
    model.setObjectiveN(gp.quicksum((m[i,0]) for i in range(state_len)),0,1, GRB.MAXIMIZE)
    model.setObjectiveN(gp.quicksum((m[i,1]) for i in range(state_len)),1,0, GRB.MINIMIZE)
    #model.setObjectiveN(gp.quicksum((m[i,1]) for i in range(state_len)), GRB.MINIMIZE)


    model.addConstrs((x[i] == (2*m[i,0] + 3*m[i,1] + 4*m[i,2])) for i in range(state_len))
    model.addConstrs(m[i,0] + m[i,1] + m[i,2] == 1 for i in range(state_len))

    #Number of recovered bit should be less than or equal to the number of keystream equations
    #model.addConstr(gp.quicksum(m[i,0] for i in range(state_len)) <= keys)
    model.addConstr(gp.quicksum(m[i,0] for i in range(state_len) if i not in linear_all) == 0)

    count = 0
    cnt = 0
    for loop in range(keys_f+keys_b):
        z = Z[loop]
        #z_var contains the index of variables in Z
        #z_var = np.array(z.variables())
        #z_var = [g.index(i) for i in z_var]
        z_monom = np.array(z.monomials())

        z_linear = [g.index(i) for i in z_monom if i.degree() == 1]

        #z_nlinear = [list(i.variables()) for i in z_monom if len(i.variables()) > 1]
        #z_nlinear = [g.index(z_nlinear[i][j]) for i in range(len(z_nlinear)) for j in range(len(z_nlinear[i]))]
        #z_nlinear = list(set(z_nlinear))


        #linear_new = [i for i in z_linear if i not in (linear + z_nlinear)]
        #z_linear_org = [i for i in z_linear if i not in z_nlinear]


        #One of the linear term variables which is not in the previous linear term has one recovered bit
        model.addConstr(gp.quicksum(m[i,0] for i in linear_new[loop]) <= 1)    
        #model.addConstr(gp.quicksum(m[i,0] for i in z_linear) <= loop + 1) 



        for mon in z_monom:
            if mon.degree() > 1:
                var = list(mon.variables())
                var = [g.index(i) for i in var]
                inters = [0]*len(linear_new)
                n = [0]*len(linear_new)
                for l in range(len(linear_new)):
                    inters[l] = intersection(var,linear_new[l])
                    n[l] = len(inters[l])

                C_prev = [n[i] for i in range(loop)] 
                C_current = [n[loop]]
                C_future = [n[i] for i in range(loop+1,keys,1)]

                #Case 1: recovered var in mon and is in linear
                #if sum(C_prev) > 0 and sum(C_current) == 0 and sum(C_future) == 0 :
                #    #if recovery found then guess
                #    model.addConstr((p[count] == 0) >> (gp.quicksum(m[i,0] for i in var) == 0))
                #    model.addConstr((p[count] == 1) >> (gp.quicksum((m[i,1] + m[i,2]) for i in var) >= len(var) - sum(C_prev)))
                #    count += 1

                #Case 2: recovered var in mon and is in z_linear-linear
                if (sum(C_current + C_future) > 0):
                    #if recovery found then fix to zero
                    prev = []
                    for k in range(loop):
                        prev = prev + inters[k]
                    #var_ge = [i for i in var if i not in prev]

                    var_ge = []
                    for k in range(loop,len(linear_new),1):
                        var_ge = var_ge + inters[k]


                    model.addConstr((q[count] == 0) >> (gp.quicksum(m[i,0] for i in var_ge) == 0))
                    model.addConstr((q[count] == 1) >> (gp.quicksum(m[i,1] for i in var) >= 1))
                    count += 1

                #if (sum(C_prev + C_current + C_future) == 0): 
                #    model.addConstr(gp.quicksum((m[i,0]) for i in var) == 0)
        #print("2nd part loop ", loop)
    #print("2nd part done")

    model.params.outputflag = 0
    start = time.time()
    model.optimize()
    end = time.time() - start


    print("Keystream = ",keys)
    print("status",model.STATUS)

    f = open(filename,'a')
    f.write("\nKeystream = " + str(keys))
    f.write("\nForward = " + str(keys_f) + "; Backward = " + str(keys_b))
    f.write("\nStatus " + str(model.STATUS) + "\n")
    f.write("\nPattern = " + str(pattern) + "\n")

    if model.STATUS == 2:
        val = model.x
        R = []
        F = []
        G = []
        for i in range(state_len):
            if round(val[i]) == 2:
                R.append(i)
            if round(val[i]) == 3:
                F.append(i)
            if round(val[i]) == 4:
                G.append(i)



        Rec = [g[i] for i in R]
        Fix = [g[i] for i in F]
        Guess = [g[i] for i in G]
        print("Recovery Bit ",Rec)
        print("Fixed Bit ",Fix)
        #print("Guessed Bit ",Guess)

        f.write(str("Recovery Bit ") + str(Rec) + "\n")
        f.write(str("Fixed Bit ") + str(Fix) + "\n")
        f.write(str("Guessed Bit ") + str(Guess) + "\n")


        print(len(R),len(F))
        if (len(R) + len(F) + len(G) == state_len) and len(intersection(R,F) + intersection(F,G) + intersection(R,G)) == 0:
            print("Length verified")
            f.write("\nLength Verified\n\n")
    end2 = time.time()-start2
    print("Gurobi Time = ", end)
    print("Overall Time = ", end2)

    f.write("\n\nGurobi Time = " + str(end))
    f.write("\nOverall Time = " + str(end2))
    f.close()



    Rec = [g[i] for i in R]
    print("Recovered Bits are ", Rec)
    Fix = [g[i] for i in F]
    print("Fixed Bits are ", Fix)

    #Assigning variables to the registers
    B = g[:90] 
    S = g[90:]

    for i in F:
        if i < 90:
            B[i] = 0
        else:
            S[i-90] = 0

    Z_f = [0]*keys_f
    Z_b = [0]*keys_b

    for loop in range(keys_b):
        S = NFSR1_bk_update(S)
        B = NFSR2_bk_update(B,S[0])

        Z_b[loop] = keystream(B,S)


    B = g[:90] 
    S = g[90:]

    #for i in F:
    #    if i < 90:
    #        B[i] = 0
    #    else:
    #        S[i-90] = 0


    for loop in range(keys_f):
        Z_f[loop] = keystream(B,S)

        if loop != keys_f-1:
            B=NFSR2_fw_update(B,S[0])
            S=NFSR1_fw_update(S) 


    index = 0
    Z = [0]*(keys_f+keys_b)

    for i in pattern:
        if i < 0:
            Z[index] = Z_b[-i-1]
        if i >= 0:
            Z[index] = Z_f[i]
        index += 1


    R_past = []
    L_rec = []
    cnt = 0
    flag = 1
    for i in range(keys_f+keys_b):
        z = Z[i]
        monom = list(z.monomials())
        t_var = list(z.variables())

        R_past = R_past + list(set(L_rec))
        m_rec = []
        print("\n",i, end = " = ")
        for m in monom:
            m_var = list(m.variables())
            if (len(m_var) == 1) and (len(intersection(m_var,Rec)) > 0):
                L_rec = L_rec + m_var
                print(intersection(m_var,Rec),end=" + ")

            if (len(m_var) > 1) and (len(intersection(m_var,Rec)) > 0):
                m_rec = m_rec + [m]
                print(intersection(m_var,Rec),"*G + ",end=" ")
        #print("monomials containing recovery bits (degree > 1) ",m_rec)

        for m in m_rec:
            m_var = list(m.variables())
            #Now L_rec contains current and past recovery bit in linear terms
            #Whereas R_past contains only the past recovery bit in linear terms
            if len(intersection(m_var,Rec)) > len(intersection(m_var,R_past)):
                f = open(filename,'a')
                f.write("\nError --- Future recovery bits found\n" + str(m))
                f.close()
                print("Error --- Future recovery bits found")
                print(m)

        L_rec = list(set(L_rec))
        if len(L_rec) != cnt + 1:
            flag = 0
        cnt = len(L_rec)
        #print("Recovery bits till now in linear terms ",L_rec)


    f = open(filename,'a')
    if flag == 0:
        print("Failed")
        f.write("\nFailed\n")
    else:
        print("Passed")
        f.write("\nPassed\n")
        with open(csv_file,'a') as f:
            writer = csv.writer(f)
            #writer.writerow(['Number of Recovered bits','Number of Fixed bits','Recovered bits','Fixed bits'])
            writer.writerow([keys_f,keys_b,len(R),len(F),Rec,Fix,pattern])
    f.close()


Pattern of the keystream bits  []
Defining b0, b1, b2, b3, b4, b5, b6, b7, b8, b9, b10, b11, b12, b13, b14, b15, b16, b17, b18, b19, b20, b21, b22, b23, b24, b25, b26, b27, b28, b29, b30, b31, b32, b33, b34, b35, b36, b37, b38, b39, b40, b41, b42, b43, b44, b45, b46, b47, b48, b49, b50, b51, b52, b53, b54, b55, b56, b57, b58, b59, b60, b61, b62, b63, b64, b65, b66, b67, b68, b69, b70, b71, b72, b73, b74, b75, b76, b77, b78, b79, b80, b81, b82, b83, b84, b85, b86, b87, b88, b89, s0, s1, s2, s3, s4, s5, s6, s7, s8, s9, s10, s11, s12, s13, s14, s15, s16, s17, s18, s19, s20, s21, s22, s23, s24, s25, s26, s27, s28, s29, s30
Keystream =  0
status 2
Recovery Bit  []
Fixed Bit  []
0 0
Length verified
Gurobi Time =  0.002324342727661133
Overall Time =  0.06662774085998535
Recovered Bits are  []
Fixed Bits are  []
Passed
Pattern of the keystream bits  [0]
Defining b0, b1, b2, b3, b4, b5, b6, b7, b8, b9, b10, b11, b12, b13, b14, b15, b16, b17, b18, b19, b20, b21, b22, b23, b24, b25, b26, b27, b28

In [14]:
for i in Z_f:
    print(i)
    
    linear = []
    nlinear = []
    
    mon = i.monomials()
    for j in mon:
        j_var = list(j.variables())
        if len(j_var) == 1:
            linear.append(j)
        else:
            nlinear.append(j)
            
    x = b0+b0
    for i in nlinear:
        x += i
    nvar = list(x.variables())
    
    for i in linear:
        print(i,end=" + ")
    print("\n")
    print(nlinear)
    print(nvar)
    print("\n\n")


b1*b19*b27*b43*b57*b66*b78 + b2*b28*b41*b65 + b4*b21 + b5 + b6*b14*b26*b32*b47*b61 + b7 + b8*b82 + b9*b52 + b11 + b13*b29*b50*b64*b75 + b18*b37 + b30 + b34*b67*b73 + b38*b63*s1*s24 + b40 + b44*b76 + b45 + b48*s9*s13 + b54 + b71 + s3*s16 + s23
b5 + b7 + b11 + b30 + b40 + b45 + b54 + b71 + s23 + 

[b1*b19*b27*b43*b57*b66*b78, b2*b28*b41*b65, b4*b21, b6*b14*b26*b32*b47*b61, b8*b82, b9*b52, b13*b29*b50*b64*b75, b18*b37, b34*b67*b73, b38*b63*s1*s24, b44*b76, b48*s9*s13, s3*s16]
[b1, b2, b4, b6, b8, b9, b13, b14, b18, b19, b21, b26, b27, b28, b29, b32, b34, b37, b38, b41, b43, b44, b47, b48, b50, b52, b57, b61, b63, b64, b65, b66, b67, b73, b75, b76, b78, b82, s1, s3, s9, s13, s16, s24]



b2*b20*b28*b44*b58*b67*b79 + b3*b29*b42*b66 + b5*b22 + b6 + b7*b15*b27*b33*b48*b62 + b8 + b9*b83 + b10*b53 + b12 + b14*b30*b51*b65*b76 + b19*b38 + b31 + b39*b64*s2*s25 + b41 + b45*b77 + b46 + b49*s10*s14 + b55 + b72 + s4*s17 + s24
b6 + b8 + b12 + b31 + b41 + b46 + b55 + b72 + s24 + 

[b2*b20*b28*b44*b58*b6

In [17]:
Rec = [b46, b54, b56, b58, b59, b74, s29]
B = g[:90] 
S = g[90:]


for i in range(keys):
    z = keystream(B,S)
    monom = list(z.monomials())
    t_var = list(z.variables())

    for m in monom:
        m_var = m.variables()
        if len(intersection(m_var,Rec)) > 0:
            print(m, end = "  ")
    
    print("\n")
    for m in monom:
        m_var = m.variables()
        if len(intersection(m_var,Rec)) > 0:
            print(intersection(m_var,Rec), end = "  ")

    B=NFSR2_fw_update(B,S[0])
    S=NFSR1_fw_update(S)
    print("\n")


b54  

[b54]  

b2*b20*b28*b44*b58*b67*b79  b35*b68*b74  b46  

[b58]  [b74]  [b46]  

b3*b21*b29*b45*b59*b68*b80  b11*b54  b46*b78  b56  

[b59]  [b54]  [b46]  [b56]  

b4*b22*b30*b46*b60*b69*b81  b74  

[b46]  [b74]  

b13*b56  b17*b33*b54*b68*b79  b58  

[b56]  [b54]  [b58]  

b7*b33*b46*b70  b43*b68*s6*s29  b59  

[b46]  [s29]  [b59]  

b15*b58  b19*b35*b56*b70*b81  b46  b54*s15*s19  s29  

[b58]  [b56]  [b46]  [b54]  [s29]  



In [12]:
print(Rec)

[b46, b54, b56, b58, b59, b74, s29]


In [16]:
z_{0} = b_{7}
z_{1} = b_{7}*G +  b_{46} 
z_{2} = b_{7} + b_{46}*G +  b_{56} 
z_{3} = b_{46}*G +  b_{7}*G +  b_{74}
z_{4} = b_{56}*G +  b_{58} 
z_{5} = b_{7}*b_{46}*G +  b_{59}  
z_{6} = b_{7}*G +  b_{58}*G +  b_{56}*G +  b_{46} + s_{29}


[b28, b68]
